In [26]:
import torch
from d2l import torch as d2l


In [27]:
#多输入通道
def corr2d_multi_in(X,K):
    #先遍历X和K的第0个维度（通道维度），再把他们加在一起
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [28]:
X = torch.tensor([[[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]],
    [[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])
K = torch.tensor([[[0.0,1.0],[2.0,3.0]],[[1.0,2.0],[3.0,4.0]]])
corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [29]:
#多输出通道
def corr2d_multi_in_out(X,K):
    #迭代K的第0个维度，每次都对输入X执行互相关运算
    #最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)
    #遍历K的第 0 维度（输出通道数），每个元素k是一个多输入通道的核（形状为 [输入通道数, 核高, 核宽]）。
    #对每个k用corr2d_multi_in(X, k)计算多输入通道到单输出通道的互相关（结果是一个二维矩阵）
    #用torch.stack(..., 0)将所有单输出通道的结果沿第 0 维度堆叠，得到多输出通道的张量（形状为 [输出通道数, 高, 宽]）


    

In [30]:
K = torch.stack((K,K+1,K+2),0)
#K+1：原核每个元素加 1，得到第二个输出通道的核
#K+2：原核每个元素加 2，得到第三个输出通道的核。
#torch.stack(..., 0)：将 3 个核沿第 0 维度堆叠，
#形状变为 [3, 2, 2, 2]（3 个输出通道，每个通道 2 个输入核，每个核 2x2）。

K.shape

torch.Size([3, 2, 2, 2])

In [31]:
corr2d_multi_in_out(X,K)
# 第 0 个输出通道：用K[0]（即原K）与X计算，结果为[[56, 72], [104, 120]]（同步骤 4 的输出）。
# 第 1 个输出通道：用K[1]（K+1）与X计算，每个通道的核元素加 1，结果为[[76, 100], [148, 180]]。
# 第 2 个输出通道：用K[2]（K+2）与X计算，每个通道的核元素加 2，结果为[[96, 128], [192, 240]]。
# 用torch.stack堆叠 3 个结果，得到形状为[3, 2, 2]的张量。

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [32]:
#6.4.3.1*1卷积层
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w = X.shape
    c_o = K.shape[0]

    # 将输入X重塑为二维矩阵：[输入通道数, 空间维度展平]（h*w是将高和宽展成一维）    
    X = X.reshape((c_i,h*w))
    # 将1x1卷积核K重塑为二维矩阵：[输出通道数, 输入通道数]（忽略1x1维度）
    K = K.reshape((c_o,c_i))
    
    # 矩阵乘法：输出通道数 × 输入通道数  乘以  输入通道数 × 空间维度展平
    #全连接层中的矩阵乘法
     # 结果形状为 [输出通道数, h*w]，等价于对每个空间位置做输入通道的线性组合
    Y = torch.matmul(K,X)

     # 将结果重塑回三维：[输出通道数, 高度, 宽度]，恢复空间结构
    return Y.reshape((c_o,h,w))

#     在 PyTorch（或 Python 中），Y.reshape((c_o, h, w)) 与 Y.reshape(c_o, h, w) 没有任何功能上的区别，两者完全等价。
# 它们的唯一差异是参数传递的形式：
# (c_o, h, w) 是一个元组（tuple），作为单个参数传递给 reshape 函数。
# c_o, h, w 是三个独立的位置参数，函数会自动将它们视为一个形状元组来处理。
#reshape 函数的参数定义允许接收一个表示形状的元组，或多个表示各维度大小的独立参

In [33]:
# 生成输入张量X：形状为(3, 3, 3)，即3个输入通道，每个通道3x3的空间尺寸
# torch.normal(0, 1, ...)表示生成均值为0、标准差为1的随机数
X = torch.normal(0,1,(3,3,3))

# 生成1x1卷积核K：形状为(2, 3, 1, 1)
# 含义：2个输出通道，每个输出通道对应3个输入通道的1x1核（最后两个维度是1x1）
K = torch.normal(0,1,(2,3,1,1))

# 用矩阵乘法实现的1x1卷积计算结果Y1
Y1 = corr2d_multi_in_out_1x1(X,K)
# 用通用的多通道互相关函数计算结果Y2（需结合之前定义的corr2d_multi_in_out）
Y2 = corr2d_multi_in_out(X,K)

# 检查Y1和Y2的差异总和是否小于1e-6（允许微小的浮点数精度误差）
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6